In [1]:
# Main notebook for playing around looking at different things in data
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
# plt.rcParams["image.cmap"] = "Dark2"
# plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
plt.rcParams['lines.markersize'] = 10
plt.rcParams['lines.linewidth'] = 3
%matplotlib widget
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import os
# import csv
import math


from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]


saveFigures is set to: True
Done loading packages


# Deaths

In [2]:
dfDeaths = pd.read_excel('DeathsAgeDK\Deaths.xlsx')
dfDeaths = dfDeaths.transpose()
dfDeaths.columns = dfDeaths.iloc[0]
dfDeaths = dfDeaths.drop(['Aldersgruppe']) 
curDates =  pd.to_datetime(dfDeaths.index,format='%d_%m_%Y')

In [3]:
# dfDeaths.diff().iloc[1:,:-1]

In [4]:
deathVals = dfDeaths.diff().iloc[1:,:-1].values.astype('int64')
deathVals = np.transpose(deathVals)

In [5]:
plt.figure()

g = sns.heatmap(deathVals,annot=True,cmap='turbo',xticklabels=curDates.strftime('%d-%b')[1:],yticklabels=dfDeaths.columns[:-1])
g.invert_yaxis()
# np.isnan(np.transpose(deathVals))

plt.title('Dødsfald')
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/DeathsHeatmap')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Indlagte

In [6]:
dfAdm = pd.read_excel('Admitted\Admitted.xlsx')
dfAdm = dfAdm.transpose()
dfAdm.columns = dfAdm.iloc[0]
dfAdm = dfAdm.drop(['Aldersgruppe']) 
curDatesAdm =  pd.to_datetime(dfAdm.index,format='%d_%m_%Y')

In [7]:
admVals = dfAdm.diff().iloc[1:,:-1].values.astype('int64')
admVals = np.transpose(admVals)

In [8]:
plt.figure()

g = sns.heatmap(admVals,annot=True,fmt="3",cmap='turbo',xticklabels=curDatesAdm.strftime('%d-%b')[1:],yticklabels=dfAdm.columns[:-1])
g.invert_yaxis()

plt.title('Nyindlæggelser')
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/AdmittedHeatmap')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Intensiv 

In [9]:
dfInt = pd.read_excel('Admitted\Intensiv.xlsx')
dfInt = dfInt.transpose()
dfInt.columns = dfInt.iloc[0]
dfInt = dfInt.drop(['Aldersgruppe']) 
curDatesInt =  pd.to_datetime(dfInt.index,format='%d_%m_%Y')

In [10]:
intVals = dfInt.diff().iloc[1:,:-1].values.astype('int64')
intVals = np.transpose(intVals)

In [11]:
plt.figure()

g = sns.heatmap(intVals,annot=True,fmt="3",cmap='turbo',xticklabels=curDatesInt.strftime('%d-%b')[1:],yticklabels=dfInt.columns[:-1])
g.invert_yaxis()

plt.title('Intensiv')
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/IntensivHeatmap')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Across the same age

In [12]:
numCols = len(dfAdm.columns)
dfAdm.columns

Index(['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
       '80-89', '90+', 'I alt'],
      dtype='object', name='Aldersgruppe')

In [13]:

# fig,ax1 = plt.subplots(1,1)
fig,allAxes = plt.subplots(numCols-1,1,sharex=True,figsize=(14,20))

colNames = dfAdm.columns
for curCol in range(0,numCols-1):

    curAx = allAxes[numCols-2-curCol]
    curAx.set_title(colNames[curCol])
    
    curDea = np.diff(dfDeaths.iloc[:,curCol].values)
    curAdm = np.diff(dfAdm.iloc[:,curCol].values)
    curInt = np.diff(dfInt.iloc[:,curCol].values)

    plotDates = curDates[1:]
    lns1 = curAx.plot(plotDates,curAdm,'b:',label='Indlæggelser')
    # curAx.plot(plotDates,curInt,label='Intensiv')
    # curAx.plot(plotDates,curDea,label='Dødsfald')
    curAx2 = curAx.twinx()
    lns2 = curAx.plot(plotDates,curInt,'k--',label='Intensiv')
    lns3 = curAx2.plot(plotDates,curDea,'r',label='Dødsfald')
    
    lns = lns1+lns2+lns3
    labs = [l.get_label() for l in lns]
        
    curAx.set_ylim(bottom=0)
    curAx2.set_ylim(bottom=0)

# allAxes[4].legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
allAxes[4].legend(lns,labs,loc='center left',bbox_to_anchor = (1.05, 0.5))

curAx.set_xlim([curDates[1],curDates[-1]])

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:

# fig,ax1 = plt.subplots(1,1)
fig,allAxes = plt.subplots(numCols-1,3,sharex=True,figsize=(14,20))

colNames = dfAdm.columns
for curCol in range(0,numCols-1):

    # curAx1 = allAxes[(numCols-2-curCol) * 3]
    # curAx2 = allAxes[(numCols-2-curCol) * 3 + 1]
    # curAx3 = allAxes[(numCols-2-curCol) * 3 + 2]
    curAx1 = allAxes[numCols-2-curCol,0]
    curAx2 = allAxes[numCols-2-curCol,1]
    curAx3 = allAxes[numCols-2-curCol,2]
    
    curAx1.set_ylabel(colNames[curCol])
    
    curDea = np.diff(dfDeaths.iloc[:,curCol].values)
    curAdm = np.diff(dfAdm.iloc[:,curCol].values)
    curInt = np.diff(dfInt.iloc[:,curCol].values)

    plotDates = curDates[1:]
    curAx1.plot(plotDates,curAdm,'b:',label='Indlæggelser')
    # curAx.plot(plotDates,curInt,label='Intensiv')
    # curAx.plot(plotDates,curDea,label='Dødsfald')
    curAx2.plot(plotDates,curInt,'k--',label='Intensiv')
    curAx3.plot(plotDates,curDea,'r',label='Dødsfald')
    
        
    curAx1.set_ylim(bottom=0)
    curAx2.set_ylim(bottom=0)
    curAx3.set_ylim(bottom=0)

# allAxes[4].legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
# allAxes[4].legend(lns,labs,loc='center left',bbox_to_anchor = (1.05, 0.5))

    
curAx1.set_xlim([curDates[0],curDates[-1]])
curAx1.set_xticks(curDates[::3])
curAx1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))

allAxes[0,0].set_title('Indlæggelser')
allAxes[0,1].set_title('Intensiv')
allAxes[0,2].set_title('Dødsfald')

plt.tight_layout()

if saveFigures:
    plt.savefig('figs/AgeDist_AllData')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:

# fig,ax1 = plt.subplots(1,1)
fig,allAxes = plt.subplots(numCols-1,3,sharex=True,figsize=(14,20))

colNames = dfAdm.columns
for curCol in range(0,numCols-1):

    # curAx1 = allAxes[(numCols-2-curCol) * 3]
    # curAx2 = allAxes[(numCols-2-curCol) * 3 + 1]
    # curAx3 = allAxes[(numCols-2-curCol) * 3 + 2]
    curAx1 = allAxes[numCols-2-curCol,0]
    curAx2 = allAxes[numCols-2-curCol,1]
    curAx3 = allAxes[numCols-2-curCol,2]
    
    curAx1.set_ylabel(colNames[curCol])
    
    curDea = np.diff(dfDeaths.iloc[:,curCol].values)
    curAdm = np.diff(dfAdm.iloc[:,curCol].values)
    curInt = np.diff(dfInt.iloc[:,curCol].values)

    plotDates = curDates[1:]

    plotDates = plotDates[13:]
    curDea = curDea[13:]
    curAdm = curAdm[13:]
    curInt = curInt[13:]


    curAx1.plot(plotDates,curAdm,'b:',label='Indlæggelser')
    # curAx.plot(plotDates,curInt,label='Intensiv')
    # curAx.plot(plotDates,curDea,label='Dødsfald')
    curAx2.plot(plotDates,curInt,'k--',label='Intensiv')
    curAx3.plot(plotDates,curDea,'r',label='Dødsfald')
    
        
    curAx1.set_ylim(bottom=0)
    curAx2.set_ylim(bottom=0)
    curAx3.set_ylim(bottom=0)

# allAxes[4].legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
# allAxes[4].legend(lns,labs,loc='center left',bbox_to_anchor = (1.05, 0.5))

    
curAx1.set_xlim([plotDates[0],plotDates[-1]])
curAx1.set_xticks(plotDates)
curAx1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))

allAxes[0,0].set_title('Indlæggelser')
allAxes[0,1].set_title('Intensiv')
allAxes[0,2].set_title('Dødsfald')

plt.tight_layout()

if saveFigures:
    plt.savefig('figs/AgeDist_AllData_Zoom')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Cases

In [16]:
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath

In [17]:
# dfCase = pd.read_csv(latestdir+'\\Cases_by_age.csv',delimiter=';')
dfCase = pd.DataFrame(columns=dfDeaths.columns)
dfTest = pd.DataFrame(columns=dfDeaths.columns)
dfPosP = pd.DataFrame(columns=dfDeaths.columns)

In [18]:
# for d in curDates:
for k in range(0,len(curDates)):
    d = curDates[k]
    curIndex = dfDeaths.index[k]

    thisDateStr = d.strftime('%Y-%m-%d')
    curDir = rootdir + '/SSI_data_' + thisDateStr 
    curFilePath = curDir + '/Cases_by_age.csv'
    curdf = pd.read_csv(curFilePath,delimiter=';',dtype=str)


    curdf['Antal_bekræftede_COVID-19'] = pd.to_numeric(curdf['Antal_bekræftede_COVID-19'].astype(str).apply(lambda x: x.replace('.','')))
    curdf['Antal_testede'] = pd.to_numeric(curdf['Antal_testede'].astype(str).apply(lambda x: x.replace('.','')))
    curdf['Procent_positive'] = pd.to_numeric(curdf['Procent_positive'].astype(str).apply(lambda x: x.replace(',','.')))

    dfCase.loc[curIndex] = curdf['Antal_bekræftede_COVID-19'].values
    dfTest.loc[curIndex] = curdf['Antal_testede'].values
    dfPosP.loc[curIndex] = curdf['Procent_positive'].values



In [19]:
dfCase

Aldersgruppe,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+,I alt
10_11_2020,2881,8543,11141,7884,8720,8486,4534,2893,1457,419,56958
17_11_2020,3383,9919,12617,8893,9870,9596,5084,3168,1570,451,64551
24_11_2020,3933,11526,14162,10030,11105,10804,5724,3509,1735,493,73021
01_12_2020,4512,13265,15830,11197,12414,12032,6365,3905,1893,536,81949
08_12_2020,5269,15977,18353,12852,14331,13767,7146,4416,2088,600,94799
15_12_2020,6548,19813,22922,15767,17416,16692,8637,5162,2425,705,116087
22_12_2020,8349,23715,27577,18985,20879,20091,10505,6341,2889,844,140175
29_12_2020,10014,26073,30632,21504,23571,22761,11986,7365,3493,1048,158447
05_01_2021,10995,27827,33147,23488,25484,24839,13348,8311,4096,1244,172779
12_01_2021,11768,29199,35144,24895,26910,26499,14348,9000,4606,1432,183801


In [20]:
caseVals = dfCase.diff().iloc[1:,:-1].values.astype('int64')
caseVals = np.transpose(caseVals)

plt.figure()

g = sns.heatmap(caseVals,annot=True,fmt="3",cmap='turbo',xticklabels=curDates.strftime('%d-%b')[1:],yticklabels=dfCase.columns[:-1], annot_kws={"size": 7})
g.invert_yaxis()

plt.title('Tilfælde')
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/CasesHeatmap')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
testVals = dfTest.diff().iloc[1:,:-1].values.astype('int64')
testVals = np.transpose(testVals)

plt.figure()

g = sns.heatmap(testVals,annot=True,fmt="3",cmap='turbo',xticklabels=curDates.strftime('%d-%b')[1:],yticklabels=dfTest.columns[:-1])
g.invert_yaxis()

plt.title('Test')
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/TestsHeatmap')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
# dfTest.diff()
# # dfCase
# # curdf
# # curdf['Antal_bekræftede_COVID-19'] = pd.to_numeric(curdf['Antal_bekræftede_COVID-19'].astype(str).apply(lambda x: x.replace('.','')))
# # curdf['Antal_testede'] = pd.to_numeric(curdf['Antal_testede'].astype(str).apply(lambda x: x.replace('.','')))
# # curdf['Procent_positive'] = pd.to_numeric(curdf['Procent_positive'].astype(str).apply(lambda x: x.replace(',','.')))
# # curdf

In [23]:
# dfTest
# # curdf['Antal_bekræftede_COVID-19'].values
# # dfDeaths.index[5]
# # dfCase.append([curdf['Antal_bekræftede_COVID-19'].values])

# Normalized by size of group

In [24]:
dfAges = pd.read_excel('DKfolketal2021clean.xlsx')

In [25]:
DK_age = []
DK_age.append(dfAges.iloc[0:10].Antal.sum())
DK_age.append(dfAges.iloc[10:20].Antal.sum())
DK_age.append(dfAges.iloc[20:30].Antal.sum())
DK_age.append(dfAges.iloc[30:40].Antal.sum())
DK_age.append(dfAges.iloc[40:50].Antal.sum())
DK_age.append(dfAges.iloc[50:60].Antal.sum())
DK_age.append(dfAges.iloc[60:70].Antal.sum())
DK_age.append(dfAges.iloc[70:80].Antal.sum())
DK_age.append(dfAges.iloc[80:90].Antal.sum())
DK_age.append(dfAges.iloc[90:].Antal.sum())
DK_age = np.array(DK_age)

In [26]:
caseValsNorm = dfCase.iloc[:,0:-1].div(DK_age).diff().iloc[1:].values.astype('float64')
caseValsNorm = 1000* np.transpose(caseValsNorm)

plt.figure()

g = sns.heatmap(caseValsNorm,annot=True,fmt="1.1f",cmap='turbo',xticklabels=curDates.strftime('%d-%b')[1:],yticklabels=dfCase.columns[:-1])
g.invert_yaxis()

plt.title('Tilfælde, per 1.000')
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/CasesHeatmapNorm')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
print(dfCase.columns[:-1])
DK_age

Index(['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
       '80-89', '90+'],
      dtype='object', name='Aldersgruppe')


array([610210, 680040, 778740, 696679, 748824, 801166, 667583, 574697,
       236648,  45458], dtype=int64)

In [28]:
deathValsNorm = dfDeaths.iloc[:,0:-1].div(DK_age).diff().iloc[1:].values.astype('float64')
deathValsNorm = 10000* np.transpose(deathValsNorm)

plt.figure()

g = sns.heatmap(deathValsNorm,annot=True,fmt="1.1f",cmap='turbo',xticklabels=curDates.strftime('%d-%b')[1:],yticklabels=dfDeaths.columns[:-1])
g.invert_yaxis()

plt.title('Dødsfald, per 10.000')
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/DeathsHeatmapNorm')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
AdmValsNorm = dfAdm.iloc[:,0:-1].div(DK_age).diff().iloc[1:].values.astype('float64')
AdmValsNorm = 10000* np.transpose(AdmValsNorm)

plt.figure()

g = sns.heatmap(AdmValsNorm,annot=True,fmt="1.1f",cmap='turbo',xticklabels=curDates.strftime('%d-%b')[1:],yticklabels=dfAdm.columns[:-1])
g.invert_yaxis()

plt.title('Indlæggelser, per 10.000')
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/AdmittedHeatmapNorm')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
IntValsNorm = dfInt.iloc[:,0:-1].div(DK_age).diff().iloc[1:].values.astype('float64')
IntValsNorm = 10000* np.transpose(IntValsNorm)

plt.figure()

g = sns.heatmap(IntValsNorm,annot=True,fmt="1.1f",cmap='turbo',xticklabels=curDates.strftime('%d-%b')[1:],yticklabels=dfInt.columns[:-1])
g.invert_yaxis()

plt.title('Intensiv, per 10.000')
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/IntensivHeatmapNorm')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Recent period

In [31]:
firstDataPoint = 13
caseValsNormRecent = dfCase.iloc[firstDataPoint:,0:-1].diff().iloc[1:].values.astype('float64')
caseValsNormRecent = np.transpose(caseValsNormRecent)


plt.figure()

g = sns.heatmap(caseValsNormRecent,annot=True,fmt="1.0f",cmap='turbo',xticklabels=curDates.strftime('%d-%b')[firstDataPoint+1:],yticklabels=dfCase.columns[:-1])
g.invert_yaxis()

plt.title('Tilfælde')
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/CasesHeatmapRecent')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
firstDataPoint = 13
caseValsNormRecent = dfCase.iloc[firstDataPoint:,0:-1].div(DK_age).diff().iloc[1:].values.astype('float64')
caseValsNormRecent = 1000* np.transpose(caseValsNormRecent)


plt.figure()

g = sns.heatmap(caseValsNormRecent,annot=True,fmt="1.1f",cmap='turbo',xticklabels=curDates.strftime('%d-%b')[firstDataPoint+1:],yticklabels=dfCase.columns[:-1])
g.invert_yaxis()

plt.title('Tilfælde, per 1.000')
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/CasesHeatmapNormRecent')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
firstDataPoint = 16
AdmValsNormRecent = dfAdm.iloc[firstDataPoint:,0:-1].div(DK_age).diff().iloc[1:].values.astype('float64')
AdmValsNormRecent = 10000* np.transpose(AdmValsNormRecent)

plt.figure()

g = sns.heatmap(AdmValsNormRecent,annot=True,fmt="1.1f",cmap='turbo',xticklabels=curDates.strftime('%d-%b')[firstDataPoint+1:],yticklabels=dfAdm.columns[:-1])
g.invert_yaxis()

plt.title('Indlæggelser, per 10.000')
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/AdmittedHeatmapNormRecent')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Normalized, graphs

In [34]:

# # fig,ax1 = plt.subplots(1,1)
# fig,allAxes = plt.subplots(numCols-1,3,sharex=True,figsize=(14,20))

# colNames = dfAdm.columns
# for curCol in range(0,numCols-1):

#     curPopSize = DK_age[curCol]

#     curAx1 = allAxes[numCols-2-curCol,0]
#     curAx2 = allAxes[numCols-2-curCol,1]
#     curAx3 = allAxes[numCols-2-curCol,2]
    
#     curAx1.set_ylabel(colNames[curCol])
    
#     curDea = 10000 * np.divide(np.diff(dfDeaths.iloc[:,curCol].values),curPopSize)
#     curAdm = 10000 * np.divide(np.diff(dfAdm.iloc[:,curCol].values),curPopSize)
#     curInt = 10000 * np.divide(np.diff(dfInt.iloc[:,curCol].values),curPopSize)

#     plotDates = curDates[1:]

#     # plotDates = plotDates[13:]
#     # curDea = curDea[13:]
#     # curAdm = curAdm[13:]
#     # curInt = curInt[13:]


#     curAx1.plot(plotDates,curAdm,'b:',label='Indlæggelser')
#     # curAx.plot(plotDates,curInt,label='Intensiv')
#     # curAx.plot(plotDates,curDea,label='Dødsfald')
#     curAx2.plot(plotDates,curInt,'k--',label='Intensiv')
#     curAx3.plot(plotDates,curDea,'r',label='Dødsfald')
    
        
#     curAx1.set_ylim(bottom=0)
#     curAx2.set_ylim(bottom=0)
#     curAx3.set_ylim(bottom=0)

# # allAxes[4].legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
# # allAxes[4].legend(lns,labs,loc='center left',bbox_to_anchor = (1.05, 0.5))

    
# curAx1.set_xlim([plotDates[0],plotDates[-1]])
# curAx1.set_xticks(plotDates)
# curAx1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))

# allAxes[0,0].set_title('Indlæggelser')
# allAxes[0,1].set_title('Intensiv')
# allAxes[0,2].set_title('Dødsfald')

# plt.tight_layout()

# # if saveFigures:
# #     plt.savefig('figs/AgeDist_AllData_Zoom')

In [35]:

fig,allAxes = plt.subplots(3,1,sharex=True)
# fig,allAxes = plt.subplots(numCols-1,3,sharex=True,figsize=(14,20))

colNames = dfAdm.columns

plotDates = curDates[1:]

# prevDea = np.zeros(plotDates.zeros)
# prevAdm = np.zeros(plotDates.zeros)
# prevInt = np.zeros(plotDates.zeros)

for curCol in range(0,numCols-1):

    curPopSize = DK_age[curCol]

    # curAx1 = allAxes[numCols-2-curCol,0]
    # curAx2 = allAxes[numCols-2-curCol,1]
    # curAx3 = allAxes[numCols-2-curCol,2]
    curAx1 = allAxes[0]
    curAx2 = allAxes[1]
    curAx3 = allAxes[2]
    
    # curAx1.set_ylabel(colNames[curCol])
    
    curDea = 10000 * np.divide(np.diff(dfDeaths.iloc[:,curCol].values),curPopSize)
    curAdm = 10000 * np.divide(np.diff(dfAdm.iloc[:,curCol].values),curPopSize)
    curInt = 10000 * np.divide(np.diff(dfInt.iloc[:,curCol].values),curPopSize)

    # plotDates = plotDates[13:]
    # curDea = curDea[13:]
    # curAdm = curAdm[13:]
    # curInt = curInt[13:]


    curAx1.plot(plotDates,curAdm,label=colNames[curCol])
    curAx2.plot(plotDates,curInt,label=colNames[curCol])
    curAx3.plot(plotDates,curDea,label=colNames[curCol])
    
    
curAx1.set_ylim(bottom=0)
curAx2.set_ylim(bottom=0)
curAx3.set_ylim(bottom=0)

curAx2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
# allAxes[4].legend(lns,labs,loc='center left',bbox_to_anchor = (1.05, 0.5))

    
curAx1.set_xlim([plotDates[0],plotDates[-1]])
curAx1.set_xticks(plotDates)
curAx1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))


allAxes[0].set_title('Indlæggelser')
allAxes[1].set_title('Intensiv')
allAxes[2].set_title('Dødsfald')

plt.tight_layout()

# if saveFigures:
#     plt.savefig('figs/AgeDist_AllData_Zoom')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:

fig,allAxes = plt.subplots(3,1,sharex=True)
# fig,allAxes = plt.subplots(numCols-1,3,sharex=True,figsize=(14,20))

colNames = dfAdm.columns

firstDataPoint = 13
# firstDataPoint = 8

plotDates = curDates[firstDataPoint+1:]
# plotDates = curDates[1:]

prevDea = np.zeros(plotDates.shape)
prevAdm = np.zeros(plotDates.shape)
prevInt = np.zeros(plotDates.shape)

for curCol in range(0,numCols-1):

    curPopSize = DK_age[curCol]

    curAx1 = allAxes[0]
    curAx2 = allAxes[1]
    curAx3 = allAxes[2]
    
    
    # curDea = 10000 * np.divide(np.diff(dfDeaths.iloc[:,curCol].values),curPopSize)
    # curAdm = 10000 * np.divide(np.diff(dfAdm.iloc[:,curCol].values),curPopSize)
    # curInt = 10000 * np.divide(np.diff(dfInt.iloc[:,curCol].values),curPopSize)
    # curDea = np.diff(dfDeaths.iloc[:,curCol].values)
    # curAdm = np.diff(dfAdm.iloc[:,curCol].values)
    # curInt = np.diff(dfInt.iloc[:,curCol].values)
    curDea = np.diff(dfDeaths.iloc[firstDataPoint:,curCol].values)
    curAdm = np.diff(dfAdm.iloc[firstDataPoint:,curCol].values)
    curInt = np.diff(dfInt.iloc[firstDataPoint:,curCol].values)




    newDea = np.add(prevDea,curDea).astype(float)
    newAdm = np.add(prevAdm,curAdm).astype(float)
    newInt = np.add(prevInt,curInt).astype(float)


    # curAx1.plot(plotDates,curAdm,label=colNames[curCol])
    # curAx2.plot(plotDates,curInt,label=colNames[curCol])
    # curAx3.plot(plotDates,curDea,label=colNames[curCol])
    # curAx1.fill_between(plotDates,curAdm,prevAdm,label=colNames[curCol])
    # curAx2.fill_between(plotDates,curInt,prevInt,label=colNames[curCol])
    # curAx3.fill_between(plotDates,curDea,prevDea,label=colNames[curCol])
    
    curAx1.fill_between(plotDates,newAdm,prevAdm,label=colNames[curCol])
    curAx2.fill_between(plotDates,newInt,prevInt,label=colNames[curCol])
    curAx3.fill_between(plotDates,newDea,prevDea,label=colNames[curCol])
    # curAx1.fill_between(plotDates,curAdm.tolist(),prevAdm,label=colNames[curCol])
    # curAx2.fill_between(plotDates,curInt.tolist(),prevInt,label=colNames[curCol])
    # curAx3.fill_between(plotDates,curDea.tolist(),prevDea,label=colNames[curCol])
    
    prevDea = newDea
    prevAdm = newAdm
    prevInt = newInt

    
curAx1.set_ylim(bottom=0)
curAx2.set_ylim(bottom=0)
curAx3.set_ylim(bottom=0)

curAx2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
# allAxes[4].legend(lns,labs,loc='center left',bbox_to_anchor = (1.05, 0.5))

    
curAx1.set_xlim([plotDates[0],plotDates[-1]])
curAx1.set_xticks(plotDates)
curAx1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))


allAxes[0].set_title('Indlæggelser')
allAxes[1].set_title('Intensiv')
allAxes[2].set_title('Dødsfald')

plt.tight_layout()

# if saveFigures:
#     plt.savefig('figs/AgeDist_AllData_Zoom')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
plt.figure()
colNames = dfAdm.columns

plotDates = curDates[1:]

prevDea = np.zeros((len(curDates)-1,))
prevAdm = np.zeros((len(curDates)-1,))
prevInt = np.zeros((len(curDates)-1,))

for curCol in range(0,numCols-1):

    curPopSize = DK_age[curCol]

    curAx1 = allAxes[0]
    curAx2 = allAxes[1]
    curAx3 = allAxes[2]
    
    
    curDea = 10000 * np.divide(np.diff(dfDeaths.iloc[:,curCol].values),curPopSize)
    curAdm = 10000 * np.divide(np.diff(dfAdm.iloc[:,curCol].values),curPopSize)
    curInt = 10000 * np.divide(np.diff(dfInt.iloc[:,curCol].values),curPopSize)

    newDea = np.add(prevDea,curDea).astype(float)

    # plt.plot(plotDates,newDea)
    plt.fill_between(plotDates,newDea,prevDea)

    prevDea = newDea

    # print(prevDea)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:

# fig,allAxes = plt.subplots(1,1,sharex=True)
fig,ax1 = plt.subplots(1,1)
# fig,allAxes = plt.subplots(numCols-1,3,sharex=True,figsize=(14,20))

colNames = dfCase.columns

firstDataPoint = 0

plotDates = curDates[firstDataPoint+1:]
# plotDates = curDates[1:]

prevCase = np.zeros(plotDates.shape)

for curCol in range(0,numCols-1):

    curPopSize = DK_age[curCol]

    curCase = np.diff(dfCase.iloc[firstDataPoint:,curCol].values)

    newCase = np.add(prevCase,curCase).astype(float)
    
    ax1.fill_between(plotDates,newCase,prevCase,label=colNames[curCol])
    
    prevCase = newCase

    
ax1.set_ylim(bottom=0)

ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))

    
ax1.set_xlim([plotDates[0],plotDates[-1]])
ax1.set_xticks(plotDates)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))


ax1.set_title('Tilfælde')

plt.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:

# fig,allAxes = plt.subplots(1,1,sharex=True)
fig,ax1 = plt.subplots(1,1)
# fig,allAxes = plt.subplots(numCols-1,3,sharex=True,figsize=(14,20))

colNames = dfCase.columns

firstDataPoint = 13

plotDates = curDates[firstDataPoint+1:]
# plotDates = curDates[1:]

prevCase = np.zeros(plotDates.shape)

for curCol in range(0,numCols-1):

    curPopSize = DK_age[curCol]

    # curCase = np.diff(dfCase.iloc[firstDataPoint:,curCol].values)
    curCase = 1000 * np.divide(np.diff(dfCase.iloc[firstDataPoint:,curCol].values),curPopSize)

    newCase = np.add(prevCase,curCase).astype(float)
    
    ax1.fill_between(plotDates,newCase,prevCase,label=colNames[curCol])
    
    prevCase = newCase

    
ax1.set_ylim(bottom=0)

ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))

    
ax1.set_xlim([plotDates[0],plotDates[-1]])
ax1.set_xticks(plotDates)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))


ax1.set_title('Tilfælde, per 1.000')

plt.tight_layout()

# if saveFigures:
#     plt.savefig('figs/AgeDist_AllData_Zoom')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …